In [1]:
import numpy as np 
from helpers import * 
from implementations import *

In [5]:
y = np.array([0.1, 0.3, 0.5])
tx = np.array([[2.3, 3.2],
       [1. , 0.1],
       [1.4, 2.3]])
initial_w = np.array([0.5, 1. ])
MAX_ITERS = 2
GAMMA = 0.1



mean_squared_error_sgd(y[:1], tx[:1], initial_w, MAX_ITERS, GAMMA)

(array([[0.0630575],
        [0.39208  ]]),
 0.8445947735940318)

In [13]:
MAX_ROWS = 5000

x_data = np.genfromtxt('data/dataset/x_train.csv', delimiter=",", skip_header=1,  max_rows=MAX_ROWS)
y_data = np.genfromtxt('data/dataset/y_train.csv', delimiter=",", skip_header=1,  max_rows=MAX_ROWS)

x_data[np.isnan(x_data)] = 0
x_data = normalize(x_data)

In [14]:
x_train, x_test, y_train, y_test = split_data(x_data, y_data, 0.7)

In [9]:
N, D = x_train.shape
initial_w = np.random.rand(D)

w, loss = logistic_regression(y_train.reshape(-1, 1), x_train, initial_w.reshape(-1, 1), 10000, 0.01)

pred = predict_logistic(x_test, w)
np.sum(pred == y_test) / len(y_test)

0.6926666666666667

In [15]:
N, D = x_train.shape
initial_w = np.random.rand(D)

w, loss = reg_logistic_regression(y_train.reshape(-1, 1), x_train, 0.5, initial_w.reshape(-1, 1), 10000, 0.01)

pred = predict_logistic(x_test, w)
np.sum(pred == y_test) / len(y_test)

0.5993333333333334

In [76]:
N, D = x_train.shape
initial_w = np.random.rand(D)

hyperparameter_tuning(x_data, y_data, reg_logistic_regression,lambdas=[0.2, 0.3], gammas=[0.01, 0.05], model_params={'initial_w': initial_w ,'max_iters': 1000})

/home/stefano/Documents/EPFL/projects/ML_project_1/implementations.py:142: RuntimeWarning: divide by zero encountered in log
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))


/home/stefano/Documents/EPFL/projects/ML_project_1/implementations.py:142: RuntimeWarning: invalid value encountered in multiply
  loss = -np.mean(y * np.log(sigmoids) + (1 - y) * np.log(1 - sigmoids))


 lambda= 0.2 gamma= 0.01 , CV accuracy = 0.5470
 lambda= 0.3 gamma= 0.01 , CV accuracy = 0.5530
 lambda= 0.2 gamma= 0.05 , CV accuracy = 0.5480
 lambda= 0.3 gamma= 0.05 , CV accuracy = 0.5620


(0.3, 0.05)